# Flooding Maps Denmark

In [2]:
import pandas as pd
from tqdm import tqdm
import os 

import rasterio
from owslib.wms import WebMapService

from climatemapping.maps import check_layer_number,download_map
from climatemapping.maps import save_merged_map

## Download data

**Right now this is an issue because we do not have access to the data at Danske Bank due to firewall protection. I've done it in my personal computer and then download it from my github account**

The code is available here in DownloadData.ipynb

https://notebooks.gesis.org/binder/v2/gh/VeraPatricio/floodingDK/master

## Data from 2019

This is for only one scenario (1/1000 years floods) but includes all Denmark

https://gis.nst.dk/server/services/ekstern/OSD_2016_2021_Plantrin1_Endelige_data/MapServer/WMSServer?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetCapabilities
 
There are several layers in this one, but the two that covers all of Denmark for an extreme flooding is

-          Fare fra hav (ekstrem hændelse)
-          Fare fra vandløb (1000 år)

In [10]:
url_2019 = 'https://gis.nst.dk/server/services/ekstern/OSD_2016_2021_Plantrin1_Endelige_data/MapServer'
wms_2019= WebMapService(url_2019+'/WMSServer')

for layer_nb in wms_2019.contents:
    print('%s %40s %s'%(layer_nb,wms_2019[layer_nb].title,wms_2019[layer_nb].crsOptions))

0                         Hovedvandoplande ['EPSG:25832', 'EPSG:4326']
1                     Vandområdedistrekter ['EPSG:25832', 'EPSG:4326']
2                           Kommunegrænser ['EPSG:25832', 'EPSG:4326']
3      Potentielt Forurenende Virksomheder ['EPSG:25832', 'EPSG:4326']
4                      Økonomisk Aktivitet ['EPSG:25832', 'EPSG:4326']
5                                Kulturarv ['EPSG:25832', 'EPSG:4326']
6                    Kritisk Infrastruktur ['EPSG:25832', 'EPSG:4326']
7                            Infrastruktur ['EPSG:25832', 'EPSG:4326']
8                                Beredskab ['EPSG:25832', 'EPSG:4326']
9                               Befolkning ['EPSG:25832', 'EPSG:4326']
10                          Arealanvendelse ['EPSG:25832', 'EPSG:4326']
11               Fare fra vandløb (1000 år) ['EPSG:25832', 'EPSG:4326']
12          Fare fra hav (ekstrem hændelse) ['EPSG:25832', 'EPSG:4326']
13                         Samlet sårbarhed ['EPSG:25832', 'EPSG:4326']
14

Download Data

In [3]:
file_name_list = []
bbox_list = []

dir_path = 'FloodingMaps/FullDK/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

file_name,bbox = download_map(url_2019,layer='11',size=(4000,4000),crs='EPSG:4326',output_dir=dir_path)
file_name_list.append(file_name)
bbox_list.append(bbox)
                                
file_name,bbox = download_map(url_2019,layer='12',size=(4000,4000),crs='EPSG:4326',output_dir=dir_path)
file_name_list.append(file_name)
bbox_list.append(bbox)   

Saved FloodingMaps/FullDK/Fare fra vandløb (1000 år).png
Saved FloodingMaps/FullDK/Fare fra hav (ekstrem hændelse).png


## Merge two maps (costal and river flooding) into one

In [4]:
bbox_merged = save_merged_map(['FloodingMaps/FullDK/Fare fra hav (ekstrem hændelse).png','FloodingMaps/FullDK/Fare fra vandløb (1000 år).png'],
                                resolution=None, # keeps the original
                                out_file='FloodingMaps/FullDK/Fare_Hav_og_Vanlob_1000ar.png')
file_name_list.append('FloodingMaps/FullDK/Fare_Hav_og_Vanlob_1000ar.png')
bbox_list.append(bbox_merged)

Save file with files names and bbox

In [5]:
df_png = pd.DataFrame({'scenario':['2019']*3,
                       'file':file_name_list,
                       'geo_zone':['DK']*3,
                       'bbox':bbox_list})
df_png.to_csv('map_list_DK_2019.csv')
df_png.tail(100)

scenario                                               file geo_zone  \
0     2019  FloodingMaps/FullDK/Fare fra vandløb (1000 år)...       DK   
1     2019  FloodingMaps/FullDK/Fare fra hav (ekstrem hænd...       DK   
2     2019  FloodingMaps/FullDK/Fare_Hav_og_Vanlob_1000ar.png       DK   

                                          bbox  
0  (8.019848, 54.445098, 15.562142, 57.653192)  
1  (8.008285, 54.435388, 15.599735, 57.769964)  
2  (8.008285, 54.435388, 15.599735, 57.769964)

## Data from 2020

The data is available in this external server: https://gis.nst.dk/server/rest/services/ekstern

and a list of the different maps available is here:

In [3]:
df_flood_maps = pd.read_csv('FloodingMaps/DK_maps_server.csv')
df_flood_maps.head(20)

zone                                                url
0       Aabenraa  https://gis.nst.dk/server/services/ekstern/Aab...
1        Esbjerg  https://gis.nst.dk/server/services/ekstern/Esb...
2     Fredericia  https://gis.nst.dk/server/services/ekstern/Fre...
3      Holstebro  https://gis.nst.dk/server/services/ekstern/Hol...
4     Juelsminde  https://gis.nst.dk/server/services/ekstern/Jue...
5   Kobenhavn_NS  https://gis.nst.dk/server/services/ekstern/Kob...
6        Kolding  https://gis.nst.dk/server/services/ekstern/Kol...
7        Korsoer  https://gis.nst.dk/server/services/ekstern/Kor...
8          Koege  https://gis.nst.dk/server/services/ekstern/Koe...
9         Nyborg  https://gis.nst.dk/server/services/ekstern/Nyb...
10        Odense  https://gis.nst.dk/server/services/ekstern/Ode...
11  RandersFjord  https://gis.nst.dk/server/services/ekstern/Ran...
12     Skaerbaek  https://gis.nst.dk/server/services/ekstern/Ska...
13    Sydlolland  https://gis.nst.dk/server/services/ekstern/Syd...
14         Vejle  https://gis.nst.dk/server/services/ekstern/Vej...
15    Vordinborg  https://gis.nst.dk/server/services/ekstern/Vor...

Each map has several layers. We can check one of them, to find the flooding maps we are interested in:

**breaks down at the office**

In [4]:
wms = WebMapService('https://gis.nst.dk/server/services/ekstern/Kobenhavn_NS_Plantrin2/MapServer/WmsServer')

for layer_nb in wms.contents:
    print('%s %40s %s'%(layer_nb,wms[layer_nb].title,wms[layer_nb].crsOptions))

1                         Risikoområde_KHS ['EPSG:25832', 'EPSG:4326']
4 Kulturarv_Seværdige_Fortidsminder_punkter_KHS ['EPSG:25832', 'EPSG:4326']
5 Kulturarv_Seværdige_Fortidsminder_areal_KHS ['EPSG:25832', 'EPSG:4326']
6                     Kulturarv_Kirker_KHS ['EPSG:25832', 'EPSG:4326']
7          Kulturarv_Fredede_Bygninger_KHS ['EPSG:25832', 'EPSG:4326']
8      Kulturarv_Fortidsminder_punkter_KHS ['EPSG:25832', 'EPSG:4326']
9        Kulturarv_Fortidsminder_areal_KHS ['EPSG:25832', 'EPSG:4326']
10                   Bes_sten_jorddiger_KHS ['EPSG:25832', 'EPSG:4326']
11                   Begravelsesomraade_KHS ['EPSG:25832', 'EPSG:4326']
13                     Vandvaerksboring_KHS ['EPSG:25832', 'EPSG:4326']
14 Miljoeinteresser_Vandforsyningsboring_KHS ['EPSG:25832', 'EPSG:4326']
15          Miljoeinteresser_Natura2000_KHS ['EPSG:25832', 'EPSG:4326']
16     Miljoeinteresser_Beskyttet_natur_KHS ['EPSG:25832', 'EPSG:4326']
17 Miljoeinteresser_Aktiv_vandindvinding_KHS ['EPSG:25832', 'E

We're going to use the layers called 'Udbredelse' something something. We need to check what is the number of this layer in each of the maps (i.e. the map of each zone)

In [5]:
scenarios = ['MTEXT_2115',
             'MTEXT_2019',
             'MT100_2115',
             'MT100_2065',
             'MT100_2019',
             'MT20_2019',
             'MT50_2115',
             'MT50_2065',
             'MT50_2019',
            ]

for scenario in scenarios:
    print(scenario)
    df_flood_maps['Udbredelse_'+scenario] = df_flood_maps.url.apply(lambda x: check_layer_number(x,'Udbredelse_\w+_'+scenario))

MTEXT_2115
MTEXT_2019
MT100_2115
MT100_2065
MT100_2019
MT20_2019
MT50_2115
MT50_2065
MT50_2019


In [ ]:
def download_map_dev(url,layer,size,crs,output_dir):
    ''' Download maps and save them as georeferenced images (i.e. with metadata refering to their coordinates)

    Parameters
    ----------

    url: string
        url to the WmsServer
    layer: int
        number of the layer to be downloaded
    size: (int,int)
       size of the image
    crs: string
        CRS of the image (check first waht is available)
   output_dir: string
       directory where to save image
   '''
   
    wms = WebMapService(url+'/WmsServer')
    srs = wms[layer].crsOptions
    bbox = wms[layer].boundingBoxWGS84 
    output_name = wms[layer].title
        
    if crs not in srs:
        print('CRS not available (available options:)',srs)
        pass
        
    # Get the image from the server
    img = wms.getmap(layers=[layer],
                     styles=['default'],
                     srs=crs,
                     bbox=bbox,
                     size=size,
                     transparent=True,
                     format='image/png8'
                    )
    
    # Save the array as an image
    print('Saved', output_dir+output_name+'.png')
    with open(output_dir+output_name+'.png', 'wb') as out:
        out.write(img.read())
       
    # Re-open the same image in rasterio to georeference it
    img_array = rasterio.open(output_dir+output_name+'.png').read().squeeze()
    img_shape = img_array.shape
    dst_transform = rasterio.transform.from_bounds(west = bbox[0],
                                                   south = bbox[1],
                                                   east = bbox[2],
                                                   north = bbox[3],
                                                   width=img_shape[1], height=img_shape[0])
    # Write it out to a file.
    with rasterio.open(output_dir+output_name+'.png', 'w', driver='GTiff',
                       width=img_shape[1], height=img_shape[0],
                       count=1, dtype=np.uint8, #nodata=0,
                       transform=dst_transform, crs=crs,
                       compress='lzw') as dst:
        dst.write(img_array.astype(np.uint8), indexes=1)
        
    return output_dir+output_name+'.png',bbox 

In [8]:
file_name_list =  []
bbox_list = []
scenario_list = []
geo_zone = []

for scenario in tqdm(scenarios):
    
    dir_path = 'FloodingMaps/Udbredelse_'+scenario+'/'
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    for idx,row in df_flood_maps.iterrows():
        if len(row['Udbredelse_'+scenario]) == 0:
            print('Scenario for ',row['Udbredelse_'+scenario],'does not exist')
            continue
        else:
            # there can be more than one layer (eg. cph north and south)
            for layer in row['Udbredelse_'+scenario]:
                file_name,bbox = download_map(row['url'],
                                               layer=layer,
                                               size=(4000,4000),
                                               crs='EPSG:4326',
                                               output_dir=dir_path)

                file_name_list.append(file_name)
                geo_zone.append(file_name.split(dir_path)[1].split(scenario)[0].split('Udbredelse_')[1].strip('_'))
                bbox_list.append(bbox)
                scenario_list.append(scenario)                

  0%|          | 0/9 [00:00<?, ?it/s]

Saved Udbredelse_MTEXT_2115/Udbredelse_AAB_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_ESB_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_FRE_MTEXT_2115.png
Scenario for  [] does not exist
Saved Udbredelse_MTEXT_2115/Udbredelse_JUE_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_KHS_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_KHN_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_KOL_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_KOR_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_KBU_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_NYB_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_ODE_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_RAF_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_SKB_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_SYD_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_NAK_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_VEJ_MTEXT_2115.png
Saved Udbredelse_MTEXT_2115/Udbredelse_V

 11%|█         | 1/9 [01:03<08:24, 63.03s/it]

Saved Udbredelse_MTEXT_2019/Udbredelse_AAB_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_ESB_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_FRE_MTEXT_2019.png
Scenario for  [] does not exist
Saved Udbredelse_MTEXT_2019/Udbredelse_JUE_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KHS_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KHN_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KOL_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KOR_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_KBU_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_NYB_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_ODE_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_RAF_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_SKB_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_SYD_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_NAK_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_VEJ_MTEXT_2019.png
Saved Udbredelse_MTEXT_2019/Udbredelse_V

 22%|██▏       | 2/9 [01:56<07:02, 60.29s/it]

...

Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Saved Udbredelse_MT50_2065/Udbredelse_HOL_MT50_2065.png


 89%|████████▉ | 8/9 [07:14<00:38, 38.66s/it]

Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Saved Udbredelse_MT50_2019/Udbredelse_HOL_MT50_2019.png


100%|██████████| 9/9 [07:16<00:00, 48.55s/it]

Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist


Save the name, bbox and scenario in a file

In [11]:
df_png = pd.DataFrame({'scenario':scenario_list,
                       'file':file_name_list,
                       'bbox':bbox_list
                      })
df_png.to_csv('map_list_DK_2020.csv')
df_png.tail(100)

scenario                                               file  \
4    MTEXT_2115  Udbredelse_MTEXT_2115/Udbredelse_KHS_MTEXT_211...   
5    MTEXT_2115  Udbredelse_MTEXT_2115/Udbredelse_KHN_MTEXT_211...   
6    MTEXT_2115  Udbredelse_MTEXT_2115/Udbredelse_KOL_MTEXT_211...   
7    MTEXT_2115  Udbredelse_MTEXT_2115/Udbredelse_KOR_MTEXT_211...   
8    MTEXT_2115  Udbredelse_MTEXT_2115/Udbredelse_KBU_MTEXT_211...   
..          ...                                                ...   
99    MT20_2019  Udbredelse_MT20_2019/Udbredelse_VEJ_MT20_2019.png   
100   MT20_2019  Udbredelse_MT20_2019/Udbredelse_VOR_MT20_2019.png   
101   MT50_2115  Udbredelse_MT50_2115/Udbredelse_HOL_MT50_2115.png   
102   MT50_2065  Udbredelse_MT50_2065/Udbredelse_HOL_MT50_2065.png   
103   MT50_2019  Udbredelse_MT50_2019/Udbredelse_HOL_MT50_2019.png   

                                             bbox  
4       (12.43739, 55.5522, 12.693495, 55.714904)  
5    (12.487415, 55.552264, 12.694708, 55.726316)  
6       (9.377683, 55.47553, 9.553077, 55.510132)  
7    (11.090517, 55.307539, 11.217802, 55.363792)  
8    (12.120189, 55.410787, 12.468536, 55.639661)  
..                                            ...  
99     (9.542685, 55.690325, 9.668523, 55.710755)  
100   (11.840523, 54.97512, 11.936832, 55.019609)  
101     (8.590139, 56.34769, 8.638287, 56.360013)  
102     (8.590139, 56.34769, 8.638287, 56.360013)  
103      (8.59072, 56.348062, 8.638312, 56.36004)  

[100 rows x 3 columns]

### Merging files

Some of the maps overlap, and this is a bit problematic further down, because a property is going to be assigned to two zones. The easiest way is to merge these overlapping maps into only one.

In [12]:
# not including the MT50
scenarios = ['MTEXT_2019',
             'MTEXT_2115',
             'MT100_2115',
             'MT100_2065',
             'MT100_2019',
             'MT20_2019']

In [13]:
def update_map_list(map_list,zone):    
    df_map_list = pd.read_csv(map_list)
    for z in drop:
        df_map_list.drop(df_map_list[df_map_list.zone == z].index,inplace=True)
    df_map_list.to_csv(map_list)

**Copenhagen region (also including Køge)**

In [15]:
for scenario in scenarios:
    save_merged_map(['FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_KHS_'+scenario+'.png',
                     'FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_KHN_'+scenario+'.png',
                      'FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_KBU_'+scenario+'.png'
                    ],
                    resolution=None, # keeps the original
                    out_file='FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_KHSN_'+scenario+'.png')
    
    update_map_list(scenario+'_map_list',['KHN','KHS','KBU'])

FileNotFoundError: [Errno 2] File MTEXT_2019_map_list does not exist: 'MTEXT_2019_map_list'

**Loland**

Loland (SLO) is split in the extreme scenarios. To simplify things, we'll merge them and call it also SLO 

In [ ]:
for scenario in ['MTEXT_2019','MTEXT_2115']:
    save_merged_map(['FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_SYD_'+scenario+'.png','FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_NAK_'+scenario+'.png'],
                resolution=None, # keeps the original
                out_file='FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_SLO_'+scenario+'.png')
    update_map_list(scenario+'_map_list',['SYD','NAK'])

**Holstebro region**

In [ ]:
for year in ['2115','2019']:
    save_merged_map(['FloodingMaps/Udbredelse_MT100_'+year+'/Udbredelse_HOL_MT100_'+year+'.png','FloodingMaps/Udbredelse_MT50_'+year+'/Udbredelse_HOL_MT50_'+year+'.png'],
                resolution=None, # keeps the original
                out_file='FloodingMaps/Udbredelse_MT50_'+year+'/Udbredelse_HOL_MT150_'+year+'.png')

## Data for Sweden

Maps are available here: https://gisapp.msb.se/Apps/oversvamningsportal/hemta-data.html

We'll be using the 200-årsflöde (Klimatanpassat flöde för slutet av seklet) and 100-årsflöde (Klimatanpassat flöde för slutet av seklet)

In [17]:
wms_swe.contents

OrderedDict([('0', <owslib.map.wms111.ContentMetadata at 0x11268e400>),
             ('1', <owslib.map.wms111.ContentMetadata at 0x11200c940>),
             ('2', <owslib.map.wms111.ContentMetadata at 0x111938160>),
             ('3', <owslib.map.wms111.ContentMetadata at 0x111938828>),
             ('4', <owslib.map.wms111.ContentMetadata at 0x111938cf8>),
             ('5', <owslib.map.wms111.ContentMetadata at 0x111938e10>),
             ('6', <owslib.map.wms111.ContentMetadata at 0x111938f28>),
             ('7', <owslib.map.wms111.ContentMetadata at 0x111941080>),
             ('8', <owslib.map.wms111.ContentMetadata at 0x111941198>),
             ('9', <owslib.map.wms111.ContentMetadata at 0x1119412b0>),
             ('10', <owslib.map.wms111.ContentMetadata at 0x1119413c8>),
             ('11', <owslib.map.wms111.ContentMetadata at 0x111941518>),
             ('12', <owslib.map.wms111.ContentMetadata at 0x1119416a0>),
             ('13', <owslib.map.wms111.ContentMetadata at 0x1

In [16]:
url_swe = 'https://gisapp.msb.se/arcgis/services/Oversvamningskarteringar/karteringar/MapServer'
wms_swe= WebMapService(url_swe+'/WMSServer')

for layer_nb in wms_swe.contents:
    print('%s %40s %s'%(layer_nb,wms[layer_nb].title,wms[layer_nb].crsOptions))

KeyError: 'No content named 0'

In [19]:
# Create path
dir_path = 'FloodingMpas/FloodingSWE2020/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# save info
file_scenario_list = []
file_name_list = []
bbox_list = []

file_name,bbox = download_map(url_swe,layer='12',size=(4000,4000),crs='EPSG:4326',output_dir='ForDash/FloodingSWE2020/')
file_scenario_list.append('200-years')
file_name_list.append(file_name)
bbox_list.append(bbox)
                                
file_name,bbox = download_map(url_swe,layer='13',size=(4000,4000),crs='EPSG:4326',output_dir='ForDash/FloodingSWE2020/')
file_scenario_list.append('100-years')
file_name_list.append(file_name)
bbox_list.append(bbox)

df_png = pd.DataFrame({'scenario':file_scenario_list,
                       'file':file_name_list,
                       'bbox':bbox_list})
df_png.to_csv('map_list_SWE_2020')
df_png.tail(100)

Saved ForDash/FloodingSWE2020/200-årsflöde (Klimatanpassat flöde för slutet av seklet).png
Saved ForDash/FloodingSWE2020/100-årsflöde (Klimatanpassat flöde för slutet av seklet).png


scenario                                               file  \
0  200-years  ForDash/FloodingSWE2020/200-årsflöde (Klimatan...   
1  100-years  ForDash/FloodingSWE2020/100-årsflöde (Klimatan...   

                                           bbox  
0   (10.424994, 55.472992, 23.711993, 67.43329)  
1  (10.255073, 55.472992, 24.032033, 68.288574)